In [ ]:
import plotly.offline as py
py.init_notebook_mode()

In [ ]:
import os, glob

from cfntem.particle_tracking.io import load_dm4_file, binarize_file_list
from cfntem.particle_tracking.image_processing import ImageEnhance, ImageMerge, ImageBinarization
from cfntem.particle_tracking.gui.image_visualization import ImageVisualizationPlotly
from cfntem.particle_tracking.gui.parameter_tuner import ImageEnhanceTuner, ImageBinarizationTuner, ImageMergeTuner

In [ ]:
img_dm4 = load_dm4_file("test_pic/Capture1_Hour_00_Minute_04_Second_11_Frame_0006.dm4")
img_plot_size = ImageVisualizationPlotly.infer_plot_size_from_image(img_dm4, 400)
half_img_plot_size = ImageVisualizationPlotly.infer_plot_size_from_image(img_dm4, 250)
vis = ImageVisualizationPlotly(img_plot_size)

In [ ]:
vis.show_image(img_dm4, img_plot_size)

In [ ]:
vis.show_histogram(img_dm4, img_plot_size)

In [ ]:
enhance_tuner = ImageEnhanceTuner(img_dm4, half_img_plot_size)
enhance_tuner.show()

In [ ]:
if not os.path.exists("bal_params.json"):
    enhance_tuner.save_params("bal_params.json")
else:
    print("preprocesing parameters file already exists, skip writting")

In [ ]:
enhance_processor = ImageEnhance("bal_params.json")
img_bal = enhance_processor.process(img_dm4)
vis.show_image(img_bal, img_plot_size)

In [ ]:
vis.show_histogram(img_bal, img_plot_size)

In [ ]:
bin_processor = ImageBinarization()
bin_tuner = ImageBinarizationTuner(img_bal, img_plot_size, bin_processor)
bin_tuner.show()

In [ ]:
if not os.path.exists("bin_params.json"):
    bin_tuner.save_params("bin_params.json")
else:
    print("binarization parameters file already exists, skip parameter tuning step")

In [ ]:
bin_processor.load_params("bin_params.json")
img_bin = bin_processor.process(img_bal)
vis.show_image(img_bin, img_plot_size)

In [ ]:
test_series = sorted(list(glob.glob("test_pic/time_series/*.dm4", recursive=True)))
merge_processor = ImageMerge()
merger_tuner = ImageMergeTuner(test_series, img_plot_size, pre_processors=[enhance_processor, bin_processor], processor=merge_processor)
merger_tuner.show()

In [ ]:
merge_processor = ImageMerge("mer_params.json")

In [ ]:
img_mer, img_marked, trust_ratio = binarize_file_list(test_series, 5, 
    processors=[enhance_processor, bin_processor, merge_processor], 
    out_file="bin_pic/merged_bin.png",
    record_file="record_pic/merged_9frames.png",
    estimate_error=True)
print("Detection Trust Ratio", trust_ratio)